In [11]:
import os
import subprocess
import time
import sys
sys.path.append("./GenomeAssemblerTA")
from contigs3 import calc_n50, longest_contig

def mkdir(path):
    try:
        os.mkdir(path)
    except Exception as e:
        #print(e)
        pass

k = "17"
results_path = r"./data/results"
path = r"./data"

my_os = "linux"
if my_os == "windows":
    velveth = r"./assemblers/Velvet_1.1.04_executables\win64\multithread\velveth_mt_x64.exe"
    velvetg = r"./assemblers/Velvet_1.1.04_executables\win64\multithread\velvetg_mt_x64.exe"
else:
    velveth = r"./assemblers/velvet_1.2.10/velveth"
    velvetg = r"./assemblers/velvet_1.2.10/velvetg"
spades = r"./assemblers/SPAdes-3.13.0-Linux/bin/spades.py"

#print(os.getcwd())

d = {'1 example.data.fasta':13,'2 synthetic.example.noerror.small.fasta':15,'3 synthetic.noerror.small.fasta':21,'4 synthetic.noerror.large.fasta':29,'5 real.error.small.fasta':35,'6 real.error.large.fasta':43}
# {'5 real.error.small.fasta':19,35,21,'6 real.error.large.fasta':53,43}

def open_file(path):
    with open(path, "r") as f:
        text = f.readlines()
        text = [t.strip() for t in text]
    return text

def run_velvet(fname, folder_with_fasta, k,assembler="spades"):
    fname = f[:-6]
    file_path = os.path.join(path,f)
    output_path = os.path.join(results_path, fname + "_{}".format(assembler))
    mkdir(output_path)
    if assembler=="velvet":
        p = subprocess.Popen([velveth, output_path, k, file_path],shell=False,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        p.wait()
        time.sleep(2)
        p = subprocess.Popen([velvetg, output_path],shell=False,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        p.wait()
        time.sleep(2)
    else:
        p = subprocess.Popen([spades, "-s", file_path, '--only-assembler', "-o", output_path],shell=False,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        #p.wait()
        #print()
        p.communicate()
        time.sleep(3)
        
    return output_path

def run_stats(velvet_path):
    if "contigs.fasta" in os.listdir(velvet_path):
        new_path = os.path.join(velvet_path, "contigs.fasta")
    else:
        new_path = os.path.join(velvet_path, "contigs.fa")
    res = open_file(new_path)
    res = "".join(["," if ">NODE" in x else x for x in res]).split(",") # combine nodes etc.
    res = [x for x in res if x!='']
    n50 = calc_n50(res)
    lng = longest_contig(res)
    summary = (f, n50, lng, len(res))
    print(summary)
    

In [5]:
import shutil
assembler = "velvet"

if False:
    for f in os.listdir(path):
        if f[-6:]==".fasta":
            print(f)
            k = str(d[f])
            output_path=run_velvet(f,path,k)

    for k in range(13,31,2):
        print("k",k)
        f = r"6 real.error.large.fasta"
        output_path = run_velvet(f,path,str(k),assembler=assembler)
        run_stats(output_path)
        try:
            shutil.rmtree(output_path)
        except:
            pass


In [12]:
assembler="spades"
if False:
    f = r"6 real.error.large.fasta"
    output_path = run_velvet(f,path,str(k),assembler=assembler)
    run_stats(output_path)

for f in os.listdir(path):
    if f[-6:]==".fasta":
        print(f)
        output_path = run_velvet(f,path,str(k),assembler=assembler)
        try:
            run_stats(output_path)
        except:
            print("No results")



1 example.data.fasta
No results
2 synthetic.example.noerror.small.fasta
No results
3 synthetic.noerror.small.fasta
No results
4 synthetic.noerror.large.fasta
('4 synthetic.noerror.large.fasta', 1134, 1226, 5)
5 real.error.small.fasta
No results
6 real.error.large.fasta
('6 real.error.large.fasta', 3897, 4239, 13)
